In [2]:
import sys
import collections
import dpkt
import socket
import os
import random
import math
import numpy as np
from scipy.stats import kurtosis, skew
from termcolor import colored
from joblib import Parallel, delayed

In [3]:
data_folder = "/Users/ct/Downloads/kinova_kortex_action_data/pcap"
dst_folder = "extractedFeatures/kinova_kortex"
DEST_IP = '192.168.1.169'
SOURCE_IP = '192.168.1.154'

# Extract UDP DATA

import processing_UDP
all_actions = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]
for action in all_actions:
    processing_UDP.FeatureExtractionCombined(data_folder, DEST_IP, SOURCE_IP, dst_folder, action)

In [4]:
all_actions = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]
actions_str = ' '.join(all_actions)

!python UDP_processor/UDP_main.py $data_folder $DEST_IP $SOURCE_IP $dst_folder $actions_str

/Users/ct/Downloads/kinova_kortex_action_data/pcap/press_key
cap_48.pcap
/Users/ct/Desktop/cybersecurity_robot/analysis_pipeline/UDP_processor/stats_comp.py:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis': kurtosis(stats),
/Users/ct/Desktop/cybersecurity_robot/analysis_pipeline/UDP_processor/stats_comp.py:12: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew': skew(stats),
cap_72.pcap
cap_3.pcap
cap_25.pcap
cap_33.pcap
cap_64.pcap
cap_13.pcap
cap_44.pcap
cap_52.pcap
cap_91.pcap
cap_29.pcap
cap_68.pcap
cap_87.pcap
cap_69.pcap
cap_86.pcap
cap_28.pcap
cap_90.pcap
cap_53.pcap
cap_45.pcap
cap_12.pcap
cap_65.pcap
cap_32.pcap
cap_24.pcap
cap_2.pcap
cap_73.pcap
cap_49.pcap
cap_15.pcap
cap_42.pcap
cap_54.pcap
cap_78.pcap
cap_97.pc

cap_161.pcap
cap_248.pcap
cap_198.pcap
cap_177.pcap
cap_82.pcap
cap_120.pcap
cap_209.pcap
cap_89.pcap
cap_66.pcap
cap_202.pcap
cap_347.pcap
cap_193.pcap
cap_31.pcap
cap_255.pcap
cap_310.pcap
cap_185.pcap
cap_27.pcap
cap_243.pcap
cap_1.pcap
cap_306.pcap
cap_70.pcap
cap_214.pcap
cap_351.pcap
cap_146.pcap
cap_280.pcap
cap_111.pcap
cap_238.pcap
cap_392.pcap
cap_107.pcap
cap_384.pcap
cap_150.pcap
cap_296.pcap
cap_279.pcap
cap_85.pcap
cap_127.pcap
cap_259.pcap
cap_170.pcap
cap_189.pcap
cap_166.pcap
cap_218.pcap
cap_93.pcap
cap_131.pcap
cap_326.pcap
cap_263.pcap
cap_371.pcap
cap_234.pcap
cap_50.pcap
cap_367.pcap
cap_222.pcap
cap_388.pcap
cap_46.pcap
cap_330.pcap
cap_275.pcap
cap_11.pcap
cap_10.pcap
cap_331.pcap
cap_274.pcap
cap_47.pcap
cap_366.pcap
cap_223.pcap
cap_389.pcap
cap_51.pcap
cap_370.pcap
cap_235.pcap
cap_327.pcap
cap_262.pcap
cap_92.pcap
cap_130.pcap
cap_219.pcap
cap_188.pcap
cap_167.pcap
cap_171.pcap
cap_258.pcap
cap_84.pcap
cap_126.pcap
cap_297.pcap
cap_278.pcap
cap_151.pcap
cap_

# Extract TCP DATA

In [ ]:
import processing_TCP
all_actions = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]
for action in all_actions:
    processing_TCP.FeatureExtractionCombined(data_folder, DEST_IP, SOURCE_IP, dst_folder, action)

# EXTRACT TEMPORAL DEPENDENCIES

In [ ]:
import os
import numpy as np
import pandas as pd
import temp_dep_extractor

# data_folder = 'size_defence_action_data/800padded'
data_folder = 'action_data'

# data_folder = "action_data"
# Get a list of all directories in the data_folder
all_actions = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]

## EXTRACT CARTESIAN COMMANDS STATISTICS

In [ ]:
kernel = np.array([154, 52, 90, 81, 52, 81, 725, 52, 81, 90, 157, 81, 154])

for action_name in all_actions:
    extraction_dir = os.path.join('extractedFeatures', action_name, 'cart')
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)

    dir_path = os.path.join(data_folder, action_name)
    files = os.listdir(dir_path)
    
    for file in files:
        if (".DS_Store" in file):
            continue
        print(file)
        pcap_file_path = os.path.join(dir_path, file)
        timestamps, packet_sizes = temp_dep_extractor.parse_pcap(pcap_file_path)
        statistics = temp_dep_extractor.extract_conv_features(timestamps, packet_sizes, file[:-5], action_name, kernel)
        df = pd.DataFrame(statistics, index=[0])
        
        output_file_path = os.path.join(extraction_dir, f'{os.path.splitext(file)[0]}.csv')
        df.to_csv(output_file_path, index=False)
        

## EXTRACT GRIPPER SPEED COMMANDS STATISTICS

In [ ]:
kernel = np.array([104, 111, 104, 111, 104, 111])

for action_name in all_actions:
    extraction_dir = os.path.join('extractedFeatures', action_name, 'gripper_speed')
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)

    dir_path = os.path.join(data_folder, action_name)
    files = os.listdir(dir_path)
    
    for file in files:
        if (".DS_Store" in file):
            continue
        print(file)
        pcap_file_path = os.path.join(dir_path, file)
        timestamps, packet_sizes = temp_dep_extractor.parse_pcap(pcap_file_path)

        # Extract features
        statistics = temp_dep_extractor.extract_corr_coeff_features(timestamps, packet_sizes, file[:-5], action_name, kernel)
        df = pd.DataFrame(statistics, index=[0])
        
        output_file_path = os.path.join(extraction_dir, f'{os.path.splitext(file)[0]}.csv')
        df.to_csv(output_file_path, index=False)


## EXTRACT GRIPPER POSITION COMMANDS STATISTICS

In [ ]:
kernel = np.array([168,66,104,95,66,104,95,66,104,390,95,104,111,95,66,95,95,66,170])
kernel = [x - 14 for x in kernel]

for action_name in all_actions:
    extraction_dir = os.path.join('extractedFeatures', action_name, 'gripper_position')
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)

    dir_path = os.path.join(data_folder, action_name)
    files = os.listdir(dir_path)
    
    for file in files:
        if (".DS_Store" in file):
            continue
        print(file)
        pcap_file_path = os.path.join(dir_path, file)
        timestamps, packet_sizes = temp_dep_extractor.parse_pcap(pcap_file_path)
        statistics = temp_dep_extractor.extract_conv_features(timestamps, packet_sizes, file[:-5], action_name, kernel)
        df = pd.DataFrame(statistics, index=[0])
        
        output_file_path = os.path.join(extraction_dir, f'{os.path.splitext(file)[0]}.csv')
        df.to_csv(output_file_path, index=False)
        

In [ ]:
res = temp_dep_extractor.custom_convolve([3,1,2,4,1,3,2,10,12,3,4,4],[2,4,1])
print(res)

# TRAIN CLASSIFIER

In [1]:
!python traffic_classifier.py "/Users/ct/Desktop/cybersecurity_robot/analysis_pipeline/extractedSTFT/kinova_kortex" press_key,route_cable,tighten_screw,pick_place,wipe_table,fold_cloth,open_drawer,pour_water --open_world_mode

Assigned labels to each class: {'fold_cloth': 0, 'open_drawer': 1, 'pick_place': 2, 'pour_water': 3, 'press_key': 4, 'route_cable': 5, 'tighten_screw': 6, 'wipe_table': 7, 'z_open_world': 8}
2
2
2
2
2
2
2
2
5
5
5
5
4
4
6
6
2
2
5
5
5
5
5
5
6
6
2
2
6
6
7
7
6
6
0
0
5
5
4
8
1
1
5
5
3
3
7
7
5
8
2
2
5
6
7
7
2
2
5
5
6
6
3
3
2
2
3
3
1
1
1
1
7
7
4
4
0
0
2
0
3
3
0
0
6
6
1
1
3
3
7
7
6
6
0
0
3
3
7
7
3
3
3
3
6
8
6
6
1
1
1
1
4
4
7
7
1
1
3
3
6
6
3
3
1
1
4
4
4
4
0
0
0
0
7
7
1
1
4
4
4
4
0
0
0
0
7
7
7
7
4
4
0
0
1
1
0
0
4
4
8
1
8
8
8
1
8
1
8
3
8
2
8
1
8
1
8
8
8
1
75
2
8
80
10
5
5
5
5
2
2
2
2
6
6
3
3
5
8
5
5
5
5
6
6
7
7
7
7
2
2
3
8
5
8
3
3
3
3
3
3
6
6
6
2
6
6
4
4
6
6
1
1
2
2
0
0
5
5
3
3
7
7
3
3
5
8
1
1
2
2
4
4
3
3
2
2
6
6
2
3
2
2
7
7
7
7
5
5
7
7
1
5
4
4
4
4
4
4
0
0
3
3
3
3
1
1
5
5
0
0
2
2
4
4
4
4
6
6
0
0
7
7
4
4
2
2
6
6
7
7
6
6
7
7
4
4
7
8
0
0
4
4
0
0
0
0
0
0
1
1
0
0
1
1
1
1
1
1
0
0
1
1
1
1
8
5
8
8
8
3
8
1
8
2
8
7
8
1
8
1
8
8
8
8
72
3
7
80
10
3
3
5
5
5
5
2
2
3
3
3
3
3
3
3
3
2
8
3
3
7
7
4
4
6
6
7
7
7
7
6
6